# <font color='#3BB9FF'>Lightweight Agentic AI: Research Assistant Agent </font>

## Abstract
In this project, I built a lightweight research agent powered by serverless LLM APIs. The goal was to automate the process of exploring a research topic — breaking it down into focused subtopics, expanding search queries, gathering information from the web, and generating a concise and refined summary.

The agent is designed around a modular workflow, where each capability — topic breakdown, query expansion, search, summarization, and critique is implemented as a standalone tool. These tools are orchestrated together in a structured agentic pipeline that simulates reasoning, reflection, and iterative improvement.

Through this project, I explored how large language models can be combined with real-world APIs to create intelligent, research-oriented assistants that operate autonomously within a defined workflow.
This serves as an introduction to broader ideas around agentic AI, autonomous tool usage, and multi-step task execution.

### Project Overview:
A modular LLM-based research agent that automates topic exploration, search, summarization, and refinement through structured reasoning and tool orchestration.


**Defining each tool as a function with the necessary parameters configurations and prompts.**
1.   Topic Breakdown Tool :  Creating a tool that takes a broad research topic
and breaks it down into smaller, more focused subtopics or subqueries. Using LLM to generate these subtopics based on the main topic.

2.  Query Expansion Tool: Developing a tool to expand the subqueries generated by the Topic Breakdown Tool. The tool should generate related keywords, synonyms, and phrases to enhance the search results.

3. Search Tool: Creating a wrapper around the Brave Search API. Please note that the free tier is 1000 queries/month. Consider creating a mock while developing, and switch to actually call the You API once the agent is more stable. Additionally, consider caching the search results.

4.  Summarizer Tool : The agent generates the summary incorporating the search results.

5. Critique Tool: Create a tool that critiques the summary, and offers suggestions of how to improve and potentially other relevant topics to search for.

6. Summarizer Tool : Create a tool that takes some input and summarizes its content using an LLM.


### Tool Implementation

In [ ]:
#Installing package
!pip install together

  Using cached pydantic-2.11.3-py3-none-any.whl.metadata (65 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached typing_extensions-4.13.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached typing_inspection-0.4.0-py3-none-any.whl.metadata (2.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 11.1 MB/s eta 0:00:00a 0:00:01
Using cached pydantic-2.11.3-py3-none-any.whl (443 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 9.1 MB/s eta 0:00:00ta 0:00:01
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
Using cached annotated_types-0.7.0-py3-none-any.whl (13 kB)
Using cached typing_extensions-4.13.2-py3-none-any.whl (45 kB)
Using cached typing_inspection-0.4.0-py3-none-any.whl (14 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.7.1
    Uninstalling typing_extensions-4.7.1:
      Successfully uninstalled typing_extensions-4.

In [ ]:
'''
Connecting TogetherAI API through API key
'''

import os
# Importing the Together library to interface with the Together API
from together import Together
# Importing userdata from Google Colab to securely store and access the API key
from google.colab import userdata

# Initialize the Together client with the API key retrieved from Google Colab's userdata
client = Together(api_key=userdata.get('TOGETHER_API_KEY'))

In [ ]:
#testing the working of model
# Creating a chat completion request using the Together API
response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo",
    messages=[
        {
                "role": "user",
                "content": "hello"
        }
],
    max_tokens=512,
    temperature=0.7,
    top_p=0.7,
    top_k=50,
    repetition_penalty=1,
    stop=["<|eot_id|>"]
)
# Printing the generated response from the model
print(response.choices[0].message.content)

Hello! How can I assist you today?


**Defining each tool as a function with the necessary parameters and descriptions.**

#### - **Topic Breakdown Tool**
This tool takes a broad research topic and uses a language model to break it down into five smaller, more focused subtopics. It helps the agent organize the research process by creating manageable and specific areas to explore, setting a clear foundation for deeper information retrieval.

  **LLM used** : mistralai/Mixtral-8x7B-Instruct-v0.1

In [ ]:
'''
Defining topic breakdown tool function
1. Topic Breakdown Tool
Creating a tool that takes a broad research topic and breaks it down into smaller, more focused subtopics or subqueries.
Using LLM to generate these subtopics based on the main topic.
'''

def topic_breakdown_tool(topic):
  response = client.chat.completions.create(
    model="mistralai/Mixtral-8x7B-Instruct-v0.1",
    messages=[
        {"role": "system", "content": "You are a helpful AI assistant."},
        {"role": "user", "content": f"Break down the following research topic into smaller 5 subtopics:{topic}"}
        ],
    max_tokens=512,
    temperature=0.7,
    top_p=0.7,
    top_k=50,
    repetition_penalty=1,
    stop=["[/INST]","</s>"],
    stream=False
    )
  return response.choices[0].message.content
#print(response.choices[0].message.content)

#### - **Query Exapnsion tool**

This tool takes the subtopics generated from the initial topic breakdown and expands them by suggesting related keywords, synonyms, and phrases. By enriching each subtopic with more search-friendly variations, it helps improve the depth and quality of information the agent can retrieve during the search phase.

In [ ]:
'''
2. Query Expansion Tool
Developing a tool to expand the subqueries generated by the Topic Breakdown Tool.
The tool should generate related keywords, synonyms, and phrases to enhance the search results.
'''

def query_expansion_tool(subtopics):
  response = client.chat.completions.create(
    model="mistralai/Mixtral-8x7B-Instruct-v0.1",
    messages=[
        {"role": "system", "content": "You are a helpful AI assistant."},
        {"role": "user", "content": f"Generate related 3 keywords, 3 synonyms and 3 phrases for all of the subtopics:{subtopics} and should follow the following format: {{'Subtopic':subtopic,'Keywords':keywords,'Synonyms':Synonyms,'Phrases':phrases}}"}
    ],
    max_tokens=512,
    temperature=0.7,
    top_p=0.7,
    top_k=50,
    repetition_penalty=1,
    stop=["[/INST]","</s>"],
    stream=False
    )
  return response.choices[0].message.content

In [ ]:
!pip install requests

- **Search Tool**

In [ ]:
#SEARCH TOOL USING YOU.COM
import requests
'''
3. Search Tool: Create a wrapper around the You API.
Please note that the free tier is 1000 queries/month.
'''
YOUR_API_KEY = userdata.get('YOU_API')
search_cache = {}

def search_tool(query):
    if query in search_cache:
        return search_cache[query]  # Return cached result
    else:
        headers = {"X-API-Key": YOUR_API_KEY}
        params = {"query": query}
        response = requests.get(f"https://api.ydc-index.io/search?query={query}",
                                params=params,
                                headers=headers,
                                ).json()
        search_cache[query] = response  # Cache the new result
        return response


#### - **Summary tool**

This tool takes the raw information gathered from the search results and transforms it into a clear, concise summary. It helps organize scattered findings into a focused, easy-to-read paragraph that captures the essence of the research topic.

In [ ]:

'''
Summarizer Tool
Creating a tool that generates the summary incorporating the search results.
'''

def summary_tool(search_results):
  response = client.chat.completions.create(
      model="mistralai/Mixtral-8x7B-Instruct-v0.1",
      messages=[
        {"role": "system", "content": "You are a helpful AI assistant."},
        {"role": "user", "content": f"Summarize the following search result:{search_results}"}
        ],
      max_tokens=512,
      temperature=0.7,
      top_p=0.7,
      top_k=50,
      repetition_penalty=1,
      stop=["[/INST]","</s>"],
      stream=False)
  return response.choices[0].message.content

#### - **Critique Tool**
This tool reviews the generated summary by evaluating its conciseness, accuracy, coherence, and completeness. It provides constructive feedback on how the summary can be improved and suggests additional areas or topics worth exploring, helping the agent refine and strengthen the final output.


In [ ]:
'''
Critique Tool
The agent critiques the summary, and improves the results.
'''

def critique_tool(summary):
  response = client.chat.completions.create(
      model="mistralai/Mixtral-8x7B-Instruct-v0.1",
      messages=[
        {"role": "system", "content": "You are a helpful AI assistant."},
        {"role": "user", "content": f"""Please critique the following summary in detail. Consider the following aspects in your critique:
        1. Conciseness: Is the summary concise and to the point without losing essential information?
        2. Accuracy: Are the facts and information in the summary accurate and well-researched?
        3. Coherence: Does the summary flow logically from one point to the next?
        4. Completeness: Does the summary cover all the key points of the topic?
        Additionally, suggest any improvements that could be made and mention any relevant topics or areas that could be further explored.
        Summary: {summary}"""}
        ],
      max_tokens=512,
      temperature=0.7,
      top_p=0.7,
      top_k=50,
      repetition_penalty=1,
      stop=["[/INST]","</s>"],
      stream=False)
  return response.choices[0].message.content

#### - **Final Summarizer Tool**
This tool refines the initial summary by incorporating feedback from the critique. It uses the suggested improvements to polish the summary, ensuring the final output is more accurate, complete, and well-structured.


In [ ]:
'''
Summarizer Tool
Creating a tool that generates the final summary using the critic results.
'''

def final_summary_tool(summary, critique):
  response = client.chat.completions.create(
      model="mistralai/Mixtral-8x7B-Instruct-v0.1",
      messages=[
        {"role": "system", "content": "You are a helpful AI assistant."},
        {"role": "user", "content": f"Improve the following summary: {summary} based on this critique suggestions: {critique}"}
        ],
      max_tokens=512,
      temperature=0.7,
      top_p=0.7,
      top_k=50,
      repetition_penalty=1,
      stop=["[/INST]","</s>"],
      stream=False)
  return response.choices[0].message.content


#### Agent Workflow

This workflow guides the research agent through a structured series of steps to transform a broad research topic into a polished, refined summary. Starting with breaking down the topic into subtopics, the agent then expands these into richer queries, searches for relevant information, and generates an initial summary. It further critiques the summary for quality and incorporates the feedback to improve the final output. By following this process, the agent ensures that the research is thorough, organized, and thoughtfully refined before presenting the final results to the user.

Workflow:
1. The agent receives a research topic from the user.
2. It uses the Topic Breakdown Tool to generate subtopics or subqueries.
3. The Query Expansion Tool expands the subqueries with related keywords and phrases.
4. The Search Tool uses the expanded queries and subqueries to gather relevant information from various sources.
5. The agent generates the summary incorporating the search results. (optional)
6. The agent critiques the summary, and improves the results. (optional)
7. The agent presents the final summary to the user.


In [ ]:
'''
Agent Workflow
Using all the tools defined above to implement an agent workflow.
Overall this entire system acts as the "Research Agent"
'''
#Agent Workflow
def research_agent(topic):
  # Tool 1: Topic Breakdown
  subtopics = topic_breakdown_tool(topic)

  # Tool 2: Query Expansion
  expanded_queries = query_expansion_tool(subtopics)

  # Tool 3: Search information
  results = search_tool(expanded_queries)
  search_results = results['hits']

  #Tool 4: Summarize the search results
  summary = summary_tool(search_results)

  #Tool 5: Critique the summary
  critique = critique_tool(summary)

  #Tool 6: Final Summarization
  final_summary_results = final_summary_tool(summary, critique)

  return {
      "subtopics": subtopics,
      "expanded_queries": expanded_queries,
      "search_results": search_results,
      "summarized_results":summary,
      "critique_results":critique,
      "final_summary":final_summary_results}



In [ ]:
# Assigning research topic to a variable and passing the to the research agent function
topic = "Future Trends and Innovations in LLM Research"
result = research_agent(topic)


In [ ]:
#Printing all the results generated from the above tools ---> working flow results of Research Agent

print (f"\033[1mRearch Topic ::{topic}\033[0m\n")
print(f"\033[1mResults from the topic_breakdown_tool ::\033[0m\n {result['subtopics']}")
print("\n")
print (f"\033[1mResults from the query_expansion_tool ::\033[0m\n {result['expanded_queries']}")
print("\n")
print (f"\033[1mResults from the search_tool ::\033[0m\n {result['search_results']}")
print("\n")
print (f"\033[1mResults from the summary_tool ::\033[0m\n {result['summarized_results']}")
print("\n")
print (f"\033[1mResults from the critique_tool ::\033[0m\n {result['critique_results']}")
print("\n")
print (f"\033[1mResults from the final_summary_tool ::\033[0m\n {result['final_summary']}")

Rearch Topic ::Future Trends and Innovations in LLM Research

Results from the topic_breakdown_tool ::
  Sure, I'd be happy to help you break down the research topic of "Future Trends and Innovations in LLM Research" into smaller subtopics. Here are five potential subtopics:

1. Advances in Natural Language Processing (NLP) Techniques: This subtopic could cover the latest developments in NLP techniques and how they might be applied to LLM research. This could include machine learning algorithms, deep learning architectures, and other computational approaches to language processing.
2. Emerging Applications of LLM in Industry: This subtopic could explore how LLM is being used in various industries, such as healthcare, finance, and education. It could also consider potential future applications of LLM in emerging fields like virtual reality and augmented reality.
3. Ethical and Social Implications of LLM: As LLM becomes more prevalent, it's important to consider the ethical and social im

## <font color='#3CB371'> **Conclusion:** </font>

In this project, I developed six modular tools — the Topic Breakdown Tool, Query Expansion Tool, Search Tool, Summary Tool, Critique Tool, and Final Summarizer Tool — each designed to handle a key step in the research process.

I then brought these tools together by creating a function that acts as a Research Agent, capable of taking a user-provided research topic and autonomously performing the entire research workflow.

Throughout the process, intermediate outputs from each tool were displayed to show how the research evolved at every stage, culminating in a polished final summary that captures the essence of the original topic.

This project also touches upon early principles of **agentic AI**, where an agent autonomously uses multiple tools, reasons through steps, reflects on its output, and refines its results — moving beyond simple single-turn LLM responses toward a more structured, goal-driven workflow.

**********************
END OF PROJECT
**********************